## Lane_detection in a video

### Importing Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

#### function to define region of interest in video 

In [2]:
def region_of_interest(img,vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask,vertices, 255)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

#### function to draw line on region of interest

In [3]:
def draw_lines(img,lines):
    line_img = np.zeros((img.shape[0],img.shape[1], 3), dtype=np.uint8)
    #if lines is not None:
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_img, (x1,y1), (x2,y2), (255,0,255), thickness=4)       
    
    img = cv2.addWeighted(img, 0.6, line_img, 1, 0)
    return img

#### function to detect lane with help of Canny edge detection and HoughLinesP modules

In [4]:
def lane_detect(img):
    height = img.shape[0]
    width = img.shape[1]
    #points = [(0,height), (width/2,330), (width,height)] #(50,130) for 'solidWhiteRight.mp4', 'solidYellowLeft.mp4' n 'test2.mp4'
    points = [(0,650), (width/4,420), (3*width/4,420), (width,650)] #(100,200) for 'lane.mp4'
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_blurr = cv2.GaussianBlur(img_gray,(5,5),0)
    img_canny = cv2.Canny(img_blurr, 100, 200, 3) 
    cropped_img = region_of_interest(img_canny, np.array([points], np.int32))
    
    lines = cv2.HoughLinesP(cropped_img, rho=2, 
                        theta=np.pi/180, 
                        threshold=50, 
                        lines=np.array([]),
                        minLineLength=50, 
                        maxLineGap=15)
    #if lines is not None:
    #    detect_lane = img
    #else:
    detect_lane = draw_lines(img,lines)
    
    return detect_lane

#### Reading input video file

In [5]:
video = cv2.VideoCapture("lane.mp4")

#### Showing detected lines on road lanes

In [6]:
while(video.isOpened()):
    ret, frame = video.read()
    frame = lane_detect(frame)
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #print(frame.shape)
    cv2.imshow('lane',frame)
    if cv2.waitKey(25) & 0XFF == ord('e'):
        break

video.release()
cv2.destroyAllWindows()
